In [2]:

import os
import re
import sqlite3
import pandas as pd
import numpy as np


## Analysis

In [3]:

def analyze_table(table_name):
    """
    Analyze a table to identify column stats and issues.
    """
    db_path = os.path.join(os.getcwd(), "../..", "data/raw/company_metadata.db")
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    print(f"Analysis for Table: {table_name}")
    print("Column Info:")
    print(df.info())
    print("\nSample Data:")
    print(df.head())
    print("-" * 50)

tables = ["company_info", "balance_sheet", "income_statement", "cash_flow", "historical_data", "analyst_recommendations"]

for table in tables:
    analyze_table(table)


Analysis for Table: company_info
Column Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2533 entries, 0 to 2532
Columns: 126 entries, address1 to bid
dtypes: float64(90), object(36)
memory usage: 2.4+ MB
None

Sample Data:
            address1                                           address2  \
0  Maker Chambers IV                        3rd Floor 222 Nariman Point   
1          TCS House  Raveline Street Corner of Hazarimal Somani Mar...   
2    HDFC Bank House       Shiv Sagar Estate Dr Annie Besant Road Worli   
3  ICICI Bank Towers                               Bandra-Kurla Complex   
4    Bharti Crescent      1, Nelson Mandela Road Vasant Kunj Phase - II   

        city      zip country            phone              fax  \
0     Mumbai   400021   India  91 22 3555 5000  91 22 2204 2268   
1     Mumbai  400 001   India  91 22 6778 9999  91 22 6778 9000   
2     Mumbai   400018   India  91 22 6652 1000             None   
3     Mumbai   400051   India  91 22 2653 6173  91

In [4]:
def find_non_numeric_rows(table_name):
    """
    Identify rows with non-numeric values in columns expected to be numeric.
    """

    def is_numeric_like(col_values):
        """
        Check if a column is numeric-like based on its values.
        A column is numeric-like if the majority of its non-null values are numeric.
        """
        non_null_values = col_values[col_values.notnull()]
        
        numeric_count = sum(
            isinstance(val, (int, float)) or 
            str(val).replace('.', '', 1).replace('-', '', 1).isdigit()
            for val in non_null_values
        )
        total_count = len(non_null_values)
        
        # Consider numeric-like if >50% of values are numeric
        return numeric_count / total_count > 0.5 if total_count > 0 else False
    
    db_path = os.path.join(os.getcwd(), "../..", "data/raw/company_metadata.db")
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    conn.close()
    
    print(f"Analyzing table: {table_name}")
    
    issues = {}
    for col in df.columns:
        if df[col].dtype == "object":  # Only analyze object columns
            # Determine if column is numeric-like
            if not is_numeric_like(df[col]):
                print(f"Skipping column: {col} (genuinely string)")
                continue
            
            # Identify rows with non-numeric values
            non_numeric_rows = df[df[col].notnull() & ~df[col].apply(lambda x: isinstance(x, (int, float)))]
            if not non_numeric_rows.empty:
                issues[col] = non_numeric_rows[[col]]

    # Display results
    if issues:
        print(f"Issues found in table: {table_name}")
        for col, rows in issues.items():
            print(f"\nColumn: {col}")
            print(rows)
    else:
        print(f"No non-numeric issues found in table: {table_name}")
        
tables = ["company_info", "balance_sheet", "income_statement", "cash_flow", "historical_data", "analyst_recommendations" ]

for table in tables:
    find_non_numeric_rows(table)


Analyzing table: company_info
Skipping column: address1 (genuinely string)
Skipping column: address2 (genuinely string)
Skipping column: city (genuinely string)
Skipping column: country (genuinely string)
Skipping column: phone (genuinely string)
Skipping column: fax (genuinely string)
Skipping column: website (genuinely string)
Skipping column: industry (genuinely string)
Skipping column: industryKey (genuinely string)
Skipping column: industryDisp (genuinely string)
Skipping column: sector (genuinely string)
Skipping column: sectorKey (genuinely string)
Skipping column: sectorDisp (genuinely string)
Skipping column: longBusinessSummary (genuinely string)
Skipping column: companyOfficers (genuinely string)
Skipping column: irWebsite (genuinely string)
Skipping column: currency (genuinely string)
Skipping column: lastSplitFactor (genuinely string)
Skipping column: exchange (genuinely string)
Skipping column: quoteType (genuinely string)
Skipping column: symbol (genuinely string)
Skippi

In [6]:
db_path = os.path.join(os.getcwd(), "../..", "data/raw/company_metadata.db")
conn = sqlite3.connect(db_path)
query = f"SELECT * FROM analyst_recommendations"
df = pd.read_sql_query(query, conn)
df.loc[3959, 'strongBuy']

0.0

## Preprocessings Utilities

In [7]:
def is_numeric_like(col_values):
        """
        Check if a column is numeric-like based on its values.
        """
        non_null_values = col_values[col_values.notnull()]
        numeric_count = sum(
            isinstance(val, (int, float)) or 
            str(val).replace('.', '', 1).replace('-', '', 1).isdigit()
            for val in non_null_values
        )
        total_count = len(non_null_values)
        return numeric_count / total_count > 0.5 if total_count > 0 else False

def is_datetime_like(col_values, threshold=0.8, date_format=None):
    """
    Check if a column is date-time-like.
    """

    def is_valid_datetime(value):
        try:
            if pd.isna(value):  # Skip NaN values
                return False
            if date_format:
                pd.to_datetime(value, format=date_format, errors="raise")
            else:
                pd.to_datetime(value, errors="raise")
            return True
        except Exception:
            return False

    # Check validity for each value in the column
    valid_count = col_values.apply(is_valid_datetime).sum()
    total_count = col_values.notnull().sum()

    # Determine if the column is date-time-like based on the threshold
    if total_count == 0:  # If all values are null, return False
        return False
    return valid_count / total_count >= threshold


### Preprocess rows with newline("\n")

In [8]:
def preprocess_latest_date_value(column):
    """
    Extract the value corresponding to the latest date from newline-separated entries.
    If no valid date-value pairs are found, return the original entry.
    
    Parameters:
        column (pd.Series): Input column with newline-separated date-value pairs.
    
    Returns:
        pd.Series: Cleaned column with values corresponding to the latest date.
    """

    def extract_latest(entry):
        if not isinstance(entry, str):
            return entry  # Return original value for non-strings
        
        # Split the entry by newline and process each line
        lines = entry.split('\n')
        date_value_pairs = []
        
        for line in lines:
            # Match date and numeric value pairs (e.g., "2024-03-31    12345.67")
            match = re.match(r"(\d{4}-\d{2}-\d{2})\s+([+-]?\d*\.?\d+|NaN)", line.strip())
            if match:
                date, value = match.groups()
                if value.lower() != "nan":  # Exclude NaN values
                    date_value_pairs.append((date, float(value)))
        
        # Sort pairs by date (latest first)
        date_value_pairs.sort(key=lambda x: x[0], reverse=True)
        
        # Return the value for the latest date, or original entry if no valid pairs
        return date_value_pairs[0][1] if date_value_pairs else entry

    # Apply the extraction logic to the entire column
    return column.apply(extract_latest)

### Preprocess nan, space and numeric values

In [9]:
def preprocess_nan_numeric_and_spaces(column):
    """
    Preprocess a column to handle:
    1. NaN values (preserve as NaN).
    2. Numeric strings (convert to int if whole number, otherwise float).
    3. Strings with spaces in numeric values (e.g., '400 001' -> 400001).
    4. Preserve non-numeric strings as is.

    Parameters:
        column (pd.Series): Input column.

    Returns:
        pd.Series: Preprocessed column.
    """

    def clean_value(value):
        if pd.isna(value):
            return np.nan  # Preserve NaN
        if isinstance(value, str):
            # Remove spaces and check if numeric
            cleaned_value = value.replace(" ", "")
            try:
                numeric_value = float(cleaned_value)
                # Check if it's a whole number to return as int
                if numeric_value.is_integer():
                    return int(numeric_value)
                return numeric_value  # Return as float otherwise
            except ValueError:
                return value  # Preserve original if not numeric
        return value  # Return original value if already numeric

    # Apply the cleaning logic to the column
    return column.apply(clean_value)

### Preprocess datetime columns

In [10]:
def preprocess_datetime_column(column, date_format=None):
        """
        Preprocess a column to handle and convert date-time-like values.

        This function performs the following operations:
        1. Valid Date-Time Strings:
        - Converts valid date-time strings into proper datetime objects.
        - If a `date_format` is provided, uses it to parse the dates.
        - If no format is provided, infers the format automatically.
        2. Invalid Entries:
        - Converts invalid or non-date values to `NaT` (Not a Time).
        3. Existing Datetime Objects:
        - Leaves existing datetime objects unchanged.

        Parameters:
            column (pd.Series): The input column to preprocess.
            date_format (str, optional): Optional date format for parsing. 
                                        Example: "%Y-%m-%d" for "2024-03-31".
                                        If None, infers the format automatically.

        Returns:
            pd.Series: Preprocessed column with:
                - Valid date-time values as `datetime64` objects.
                - Invalid or non-date values as `NaT`.
        """
        def convert_to_datetime(value):
            try:
                if date_format:
                    return pd.to_datetime(value, format=date_format, errors="coerce")
                else:
                    return pd.to_datetime(value, errors="coerce")
            except Exception:
                return pd.NaT  # Return NaT for invalid entries

        # Apply the conversion logic
        return column.apply(convert_to_datetime)


## Validation

In [11]:
def validate_post_preprocessing_for_numeric(column):
    """
    Validate a column after preprocessing to ensure no non-numeric values remain.

    Parameters:
        column (pd.Series): The input Pandas column.

    Returns:
        pd.DataFrame: Rows that are still problematic (non-numeric).
    """
    # Check for non-numeric values
    problematic_rows = column[~column.apply(lambda x: isinstance(x, (int, float)) or pd.isnull(x))]
    
    if not problematic_rows.empty:
        print(f"Validation Failed: Found {len(problematic_rows)} non-numeric rows.")
        return problematic_rows
    else:
        print("Validation Passed: All values are numeric or null.")
        return pd.DataFrame()  # Empty DataFrame if no issues found

def validate_post_preprocessing_for_datetime(column):
    """
    Validate a column after preprocessing to ensure all values are valid datetime objects or NaT.

    Parameters:
        column (pd.Series): The input Pandas column.

    Returns:
        pd.DataFrame: Rows that are still problematic (non-datetime).
    """

    # Check for non-datetime values
    problematic_rows = column[~column.apply(lambda x: pd.isnull(x) or isinstance(x, pd.Timestamp))]
    
    if not problematic_rows.empty:
        print(f"Validation Failed: Found {len(problematic_rows)} non-datetime rows.")
        return problematic_rows
    else:
        print("Validation Passed: All values are valid datetime or null.")
        return pd.DataFrame()  # Empty DataFrame if no issues found

In [13]:
def validate_numeric_conversion(table_name):
    """
    Validate numeric-like columns in a table post-preprocessing.
    Flags rows in numeric-like columns that are not numeric and displays
    3 sample rows for each numeric-like column after preprocessing.
    """

    db_path = os.path.join(os.getcwd(), "../..", "data/raw/company_metadata.db")
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    conn.close()

    print(f"Validating table: {table_name}")

    flagged_rows = {}
    for col in df.columns:
        if df[col].dtype == "object":  # Only consider object columns
            if is_numeric_like(df[col]):
                print(f"Validating column: {col} (numeric-like)")

                # Preprocess rows with '\n'
                if df[col].str.contains('\n', na=False).any():
                    print(f"Preprocessing column: {col} (contains rows with '\\n')")
                    df[col] = preprocess_latest_date_value(df[col])
                
                # Preprocess NaN and numeric strings
                df[col] = preprocess_nan_numeric_and_spaces(df[col])
                
                # Validate the column
                problematic_rows = validate_post_preprocessing_for_numeric(df[col])
                
                # If no issues, display 3 sample rows
                if problematic_rows.empty:
                    print(f"Sample rows from column: {col}")
                    print(df[col].head(3))
                else:
                    flagged_rows[col] = problematic_rows

    # Display flagged rows if any issues remain
    if flagged_rows:
        print("\nFlagged Rows in Numeric-Like Columns:")
        for col, rows in flagged_rows.items():
            print(f"Column: {col}")
            print(rows)
    else:
        print("All numeric-like columns are correctly converted.")

def validate_datetime_conversion(table_name):
    """
    Validate datetime-like columns in a table post-preprocessing.
    Flags rows in datetime-like columns that are not valid datetime values and displays
    3 sample rows for each datetime-like column after preprocessing.
    """

    db_path = os.path.join(os.getcwd(), "../..", "data/raw/company_metadata.db")
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    conn.close()

    print(f"Validating table: {table_name}")

    flagged_rows = {}
    for col in df.columns:
        if df[col].dtype == "object":  # Only consider object columns
            # Detect if the column is datetime-like
            if is_datetime_like(df[col]):
                print(f"Validating column: {col} (datetime-like)")

                # Preprocess the column
                df[col] = preprocess_datetime_column(df[col])
                
                # Validate the column
                problematic_rows = validate_post_preprocessing_for_datetime(df[col])
                
                # If no issues, display 3 sample rows
                if problematic_rows.empty:
                    print(f"Sample rows from column: {col}")
                    print(df[col].head(3))
                else:
                    flagged_rows[col] = problematic_rows

    # Display flagged rows if any issues remain
    if flagged_rows:
        print("\nFlagged Rows in Datetime-Like Columns:")
        for col, rows in flagged_rows.items():
            print(f"Column: {col}")
            print(rows)
    else:
        print("All datetime-like columns are correctly converted.")

In [14]:
tables = ["company_info", "balance_sheet", "income_statement", "cash_flow", "historical_data", "analyst_recommendation"]

for table in tables:
    validate_numeric_conversion(table)
    validate_datetime_conversion(table)


Validating table: analyst_recommendations
Validating column: strongBuy (numeric-like)
Preprocessing column: strongBuy (contains rows with '\n')
Validation Failed: Found 1 non-numeric rows.
Validating column: buy (numeric-like)
Preprocessing column: buy (contains rows with '\n')
Validation Failed: Found 1 non-numeric rows.
Validating column: hold (numeric-like)
Preprocessing column: hold (contains rows with '\n')
Validation Failed: Found 1 non-numeric rows.
Validating column: sell (numeric-like)
Preprocessing column: sell (contains rows with '\n')
Validation Failed: Found 1 non-numeric rows.
Validating column: strongSell (numeric-like)
Preprocessing column: strongSell (contains rows with '\n')
Validation Failed: Found 1 non-numeric rows.
Validating column: ticker_id (numeric-like)
Validation Passed: All values are numeric or null.
Sample rows from column: ticker_id
0    3801970844685813760
1    3801970844685813760
2    3801970844685813760
Name: ticker_id, dtype: int64

Flagged Rows in N